<a href="https://colab.research.google.com/github/fedex2118/GarbageDetection/blob/main/True_Garbage_Yolo5_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Link to the test data
https://drive.google.com/drive/folders/1qbHHrTfJY8v4kR5zwCRJQ5J9I2NhX0kd?usp=sharing


In [ ]:
!pip install numpy

#Install Dependencies

In [ ]:
!conda install git

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 127.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
# install dependencies as necessary
%cd /content/yolov5
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
#from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

This is the for the original dataset used to train Yolov5

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
from roboflow import Roboflow
rf = Roboflow(api_key="raifJZV5ydW0w0mCsbV3")
project = rf.workspace("material-identification").project("garbage-classification-3")
version = project.version(2)
dataset = version.download("yolov5")



loading Roboflow workspace...
loading Roboflow project...


This is the for the new dataset used to train yolov5

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="fjcIAf6clQ73LrRJ4kTZ")
project = rf.workspace("yolo-lvle4").project("waste-fx8bs")
version = project.version(5)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to waste-5 in yolov5pytorch:: 100%|██████████████| 19586/19586 [00:30<00:00, 650.62it/s]


#Define Model and Architecture

In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml   #Insert path to the local yaml file

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

#Train Model

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
# Use the right path based on where you are working
%%time
#%cd /content/yolov5/

# Workers 0 may be needed in order to work
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_results  --cache  --workers 0

Same here but for the new dataset

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
#%cd /content/yolov5/
!python C:\Users\Tommaso\TrashDetectorProject\yolov5\train.py --img 640 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --cfg C:\Users\Tommaso\TrashDetectorProject\yolov5\models\yolov5s.yaml --weights yolov5s.pt --name yolov5s_results  --cache --workers 0

#Run Inference with Trained Weights

In [ ]:
#Paths should be adapted as usual based on location and folder with weights
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results7/weights/best.pt --img 416 --source "/content/gdrive/My Drive/Garbage Yolo/test" --save-txt --save-conf